In [32]:
pip install numpy tensorflow-cpu timm sklearn

Defaulting to user installation because normal site-packages is not writeable
  Using cached timm-0.9.11-py3-none-any.whl.metadata (60 kB)
  Using cached sklearn-0.0.post11.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - a

In [33]:
import argparse
import os 
import time
import datasets
#import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.models import Model
from sklearn.metrics import classification_report
from tensorflow.keras import regularizers

ModuleNotFoundError: No module named 'tensorflow'

In [7]:
array = list(range(10))

In [9]:
np.save('array.npy', array)

In [10]:
hej = np.load('array.npy')

In [12]:
pip install datasets[vision]

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 8.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 1.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 43.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 25.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.0 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 12.9 MB/s eta 

In [14]:
import datasets

In [24]:
import os
ds_train = datasets.load_from_disk(os.path.join("..", "datasets", 'eva02_base_train'))
ds_train = ds_train[0:100]
ds_train.save_to_disk(os.path.join('..', 'datasets', "plott_test_train"))


AttributeError: 'dict' object has no attribute 'save_to_disk'

In [26]:
type(ds_train)

dict

In [23]:
ds_val = datasets.load_from_disk(os.path.join("..", "datasets", 'eva02_base_val'))
ds_val = ds_val[0:100]
ds_val.save_to_disk(os.path.join('..', 'datasets', "plott_test_val"))

AttributeError: 'dict' object has no attribute 'save_to_disk'

In [22]:
ds_test = datasets.load_from_disk(os.path.join("..", "datasets", 'eva02_base_test'))
ds_test = ds_test[0:100]
ds_test.save_to_disk(os.path.join('..', 'datasets', "plott_test_test"))

100

In [ ]:
def fit_and_predict(train_data, test_data, val_data, hidden_layer_size, embedding_col, feature_col, batch_size, epochs):

    '''fit a compiled model on training data and predict on test dataset'''

    model = build_classfication_model(train_data, hidden_layer_size, feature_col, embedding_col)

    # convert to tensorflow datasets
    tf_ds_train = train_data.to_tf_dataset(
            columns=embedding_col, # the columns to be used as inputs to the model, X
            label_cols=feature_col, # columns containing class labels, y
            batch_size=batch_size,
            shuffle=True
            )
    
    tf_ds_test = test_data.to_tf_dataset(
            columns=embedding_col,
            label_cols=feature_col, 
            batch_size=batch_size,
            shuffle=False # for test data, set shuffle to false
            )
    
    tf_ds_val = val_data.to_tf_dataset(
            columns=embedding_col,
            label_cols=feature_col, 
            batch_size=batch_size,
            shuffle=False # ?
            )

    # define steps per epoch
    #epoch_steps = (len(train_data) // batch_size) -1
    #print(epoch_steps)

    # add early stopping, stop training if validation loss does not improve for three epochs
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    # start timer
    start_time = time.time()

    # fit model and save history
    H = model.fit(tf_ds_train, epochs = epochs, validation_data=tf_ds_val, callbacks=[early_stopping])

    end_time = time.time() - start_time

    # save time as txt
    with open(f"out/times/{embedding_col}_{feature_col}_training_l2.txt", "w") as f:
       f.write(str(end_time))

    num_epochs= len(H.history['val_loss'])
    print(f"Model ran for {num_epochs} epochs")

    # save history plot in "plots" folder
    save_plot_history(H, num_epochs, f'{embedding_col}_{feature_col}_history_l2.png')

    # predict on test data
    predictions = model.predict(tf_ds_test)

    # find class with the highest probability
    predicted_classes = np.argmax(predictions,axis=1)

    # save predicted classes as .npy to be used for plotting
    np.save(f'out/y_pred/{embedding_col}_{feature_col}_y_pred.npy', predicted_classes)

    return predicted_classes